Assess Spaceship Titanic with CredoAI - Lens
============================================

In [1]:
import pandas as pd
import joblib

import warnings
warnings.filterwarnings('ignore')

from credoai.lens import Lens
from credoai.artifacts import ClassificationModel, TabularData
from credoai.evaluators import ModelFairness, Performance

# For this to work, you need to "File / Save and export notebook as... / Executable Script" the notebook
import Spaceship_Titanic_data

(train) Number of rows = 8693 and Number of cols = 14
(test) Number of rows = 4277 and Number of cols = 13


Load trained model
------------------


In [2]:
feature_engineering = Spaceship_Titanic_data.feature_engineering

with open('model.jlb', 'rb') as file:
    model = joblib.load(file)
    
processor = model.steps[0][1]
drop_target = model.steps[1][1]
classifier = model.steps[2][1]

processor

Pipeline(steps=[('fe_eng',
                 FunctionTransformer(func=<function feature_engineering at 0x7fbfedac0040>)),
                ('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('label_imputer',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['HomePlanet', 'CryoSleep',
                                                   'Destination', 'VIP',
                                                   'PassengerGGG',
                                                   'PassengerPP']),
                                                 ('num_imputer',
                                                  SimpleImputer(),
                                                  ['Age', 'RoomService...
                                   transformers=[('std_scaler',
                                                  StandardScaler(),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck']),
                                                 ('minmax_scaler',
                                                  MinMaxScaler(feature_range=(-1,
                                                                              1)),
                                                  ['PassengerGGG',
                                                   'PassengerPP']),
                                                 ('one_hot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                sparse_output=False),
                                                  ['HomePlanet', 'CryoSleep',
                                                   'Destination', 'VIP'])],
                                   verbose_feature_names_out=False))])

In [3]:
transformed_data = processor.fit_transform(Spaceship_Titanic_data.train_data)
dropped_data = drop_target.fit_transform(transformed_data)

In [4]:
ages = pd.qcut(transformed_data.Age, 10)

In [5]:
credo_model = ClassificationModel(name="titanic_default_classifier",
                                  model_like=classifier)
credo_data = TabularData(
    name="titanic-default",
    X=dropped_data,
    y=transformed_data.Transported,
    sensitive_features=ages
)

lens = Lens(model=credo_model, assessment_data=credo_data)
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.add(Performance(metrics=metrics))
lens.run()

2023-04-19 09:59:40,347 - lens - INFO - Evaluator ModelFairness added to pipeline. Sensitive feature: Age
2023-04-19 09:59:40,392 - lens - INFO - Evaluator Performance added to pipeline. 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s finished


In [6]:
results = lens.get_results()
print(f"Results for {len(results)} evaluators")

Results for 2 evaluators


In [7]:
performance_results = lens.get_results(evaluator_name='Performance')[0]
# first dataframe is overall metrics
display(performance_results['results'][0])
# second dataframe is the long form of the confusion matrix
display(performance_results['results'][1])

,type,value
0,precision_score,0.790921
1,recall_score,0.827775


,true_label,predicted_label,value
0,False,False,0.777984
1,True,False,0.172225
2,False,True,0.222016
3,True,True,0.827775
